In [4]:
#! pip install tensorflow tensorflow-hub

In [58]:
#! pip install kaggle

In [79]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path

# dataset

In [63]:
#! kaggle datasets download -d safwanshamsir99/solar-photovoltaics-panell-for-dust-dectection

In [64]:
# import zipfile
# import os

# # Define the path to your zip file and the extraction directory
# zip_file_path = '/workspaces/solar_panels/solar-photovoltaics-panell-for-dust-dectection.zip'
# extraction_directory = '/workspaces/solar_panels/dataset'

# # Create the directory where you want to extract the files, if it does not exist
# if not os.path.exists(extraction_directory):
#     os.makedirs(extraction_directory)

# # Extract the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extraction_directory)

# print(f"Files extracted to: {extraction_directory}")

Files extracted to: /workspaces/solar_panels/dataset


# preprocessing

In [66]:
img_generator = tf.keras.preprocessing.image.ImageDataGenerator(#rotation_range=90,
                                                                brightness_range=(0.5,1), 
                                                                #shear_range=0.2, 
                                                                #zoom_range=0.2,
                                                                channel_shift_range=0.2,
                                                                horizontal_flip=True,
                                                                vertical_flip=True,
                                                                rescale=1./255,
                                                                validation_split=0.3)

In [70]:
img_generator_flow_train = img_generator.flow_from_directory(
    directory="/workspaces/solar_panels/dataset/dataset",
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    subset="training")

img_generator_flow_valid = img_generator.flow_from_directory(
    directory="/workspaces/solar_panels/dataset/dataset",
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    subset="validation")

Found 590 images belonging to 2 classes.
Found 252 images belonging to 2 classes.


In [77]:
imgs, labels = next(iter(img_generator_flow_train))

In [ ]:
for img, label in zip(imgs, labels):
  true_file_path = np.argmax(img_generator_flow_train.labels == tf.argmax(label))
  true_label_name = Path(img_generator_flow_train.filepaths[true_file_path]).parent.name
  print("Label :", true_label_name)
  plt.imshow(img)
  plt.show()

# remove last layer and freeze the rest

In [71]:
base_model = tf.keras.applications.MobileNet(input_shape=(224,224,3), 
                                               include_top=False,
                                               weights = "imagenet"
                                               )

In [72]:
base_model.trainable = False

In [73]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation='softmax')  # Adjust for your binary classification
])

In [74]:
# Let's create a learning rate schedule to decrease the learning rate as we train the model. 
initial_learning_rate = 0.001

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True)


In [75]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = lr_schedule),
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = [tf.keras.metrics.CategoricalAccuracy()])

In [76]:
model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional) │ ?                      │     3,228,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,228,864 (12.32 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 3,228,864 (12.32 MB)